In [1]:
import os
import csv
import psycopg2
import mysql
import MySQLdb
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, DATE, TIMESTAMP, VARCHAR, FLOAT, INTEGER, select, text
import sqlalchemy 
from psycopg2 import sql
import pandas as pd
import create_table_leanx as ctl
import table_schemas

In [ ]:
db_host = 'academymysql.cluster-ckynnbglhixw.eu-central-1.rds.amazonaws.com'
db_port = '3306'
db_name = 'ocpm_om_test'
db_user = 'admin'
db_password = ''

# Construct the connection string
connection_string = sql.SQL("dbname={} user={} password={} host={} port={}").format(
    sql.Identifier(db_name),
    sql.Identifier(db_user),
    sql.Identifier(db_password),
    sql.Identifier(db_host),
    sql.Identifier(db_port)
)

In [3]:
connection = MySQLdb.connect(db=db_name, user=db_user, passwd=db_password, host=db_host)
cursor = connection.cursor()
root_folder = "data/OCPM/"
engine = create_engine(f"mysql+mysqldb://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# lst of column names which needs to be string
lst_str_cols = ['ABGRU','BSCHL','POSNR','VBELN', 'OBJNR','BELNR', 'MBLNR', 'CHANGENR', 'OBJECTCLAS', 'OBJECTID', 'CDTCODE', 'CHGNR', 'AWKEY', 'VBELV', 'BSTNK', 'VGBEL', 'VBELN_IM','FAKSP', 'FAKSK','LIFSK', 'LIFSP','FAKSD']
# use dictionary comprehension to make dict of dtypes
dict_dtypes = {x : 'str'  for x in lst_str_cols}

ts = table_schemas.table_schemas
ts_csv = table_schemas.columns_to_parse
primarykeys = table_schemas.primekeys

def find_csv_files(root_folder):
    csv_files = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
    return csv_files

def create_or_overwrite_table(csv_file_path, cursor):
    table_name = csv_file_path.split('\\')[-1]
    table_name = table_name.split('.')[0]
    print(f"Creating or overwriting table: {table_name} with data from: {csv_file_path}")

    df = pd.read_csv(csv_file_path, parse_dates=ts_csv[table_name], date_format="mixed", dtype=dict_dtypes) # try date_format here 
    #engine = create_engine(f"mysql+mysqldb://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    df.to_sql(name=table_name, con=engine, if_exists='replace', index=False, dtype=ts[table_name])

    with engine.connect() as conn:
        query = f'ALTER TABLE {table_name} ADD CONSTRAINT Prime PRIMARY KEY ({primarykeys[table_name]});'
        conn.execute(text(query))

csv_files = find_csv_files(root_folder)
for csv_file in csv_files:
    create_or_overwrite_table(csv_file, cursor)

cursor.close()
connection.close()

Creating or overwriting table: ADRP with data from: data/OCPM/master\ADRP.csv
Creating or overwriting table: KNA1 with data from: data/OCPM/master\KNA1.csv
Creating or overwriting table: KNB1 with data from: data/OCPM/master\KNB1.csv
Creating or overwriting table: KNKK with data from: data/OCPM/master\KNKK.csv
Creating or overwriting table: KNVV with data from: data/OCPM/master\KNVV.csv
Creating or overwriting table: LFA1 with data from: data/OCPM/master\LFA1.csv
Creating or overwriting table: LFB1 with data from: data/OCPM/master\LFB1.csv
Creating or overwriting table: MAKT with data from: data/OCPM/master\MAKT.csv
Creating or overwriting table: MARA with data from: data/OCPM/master\MARA.csv
Creating or overwriting table: MARC with data from: data/OCPM/master\MARC.csv
Creating or overwriting table: MARM with data from: data/OCPM/master\MARM.csv
Creating or overwriting table: MBEW with data from: data/OCPM/master\MBEW.csv
Creating or overwriting table: S067 with data from: data/OCPM/ma